In [1]:
!pip install rasterio

In [4]:
import os
import numpy as np
import rasterio

# Diretório contendo os arquivos raster
diretorio = r"E:\Process"

# Função para aplicar a regra de comparação entre dois arrays
def comparar_rasters(arr1, arr2):
    resultado = np.zeros_like(arr1, dtype=np.uint8)
    resultado[(arr1 == 1) & (arr2 == 1)] = 1
    resultado[(arr1 == 1) & (arr2 == 0)] = 2
    resultado[(arr1 == 0) & (arr2 == 1)] = 3
    return resultado

# Loop pelos anos de 1985 a 2022 para comparar com o ano seguinte
for ano in range(2014, 2016):
    arquivo1 = os.path.join(diretorio, f"base{ano}.tif")
    arquivo2 = os.path.join(diretorio, f"base{ano+1}.tif")
    saida = os.path.join(diretorio, f"comparacao_{ano}_{ano+1}.tif")

    print(f"Comparando {arquivo1} com {arquivo2}...")

    with rasterio.open(arquivo1) as src1, rasterio.open(arquivo2) as src2:
        arr1 = src1.read(1)
        arr2 = src2.read(1)
        perfil = src1.profile

        resultado = comparar_rasters(arr1, arr2)

        perfil.update(dtype=rasterio.uint8, count=1, nodata=255)

        with rasterio.open(saida, 'w', **perfil) as dst:
            dst.write(resultado, 1)

    print(f"Arquivo de saída gerado: {saida}")

print("Processamento concluído.")


Comparando E:\Process\base2014.tif com E:\Process\base2015.tif...
Arquivo de saída gerado: E:\Process\comparacao_2014_2015.tif
Comparando E:\Process\base2015.tif com E:\Process\base2016.tif...
Arquivo de saída gerado: E:\Process\comparacao_2015_2016.tif
Processamento concluído.


In [1]:
!pip install rasterio scipy


In [5]:
import os
import numpy as np
import rasterio
from scipy.ndimage import label

# Diretório contendo os arquivos de comparação
diretorio = r"E:\Process"

# Estrutura de conectividade rook (4-vizinhos)
structure = np.array([[0, 1, 0],
                      [1, 1, 1],
                      [0, 1, 0]])

# Função para salvar raster com base em uma máscara
def salvar_raster_mascara(mascara, perfil, caminho_saida):
    perfil.update(dtype=rasterio.uint8, count=1, nodata=0)
    with rasterio.open(caminho_saida, 'w', **perfil) as dst:
        dst.write(mascara.astype(np.uint8), 1)

# Processar todos os arquivos de comparação
for nome_arquivo in os.listdir(diretorio):
    if nome_arquivo.startswith("comparacao_") and nome_arquivo.endswith(".tif"):
        caminho = os.path.join(diretorio, nome_arquivo)
        print(f"Processando {nome_arquivo}...")

        with rasterio.open(caminho) as src:
            dados = src.read(1)
            perfil = src.profile

            # Extrair anos do nome do arquivo
            partes = nome_arquivo.replace(".tif", "").split("_")
            ano1, ano2 = partes[1], partes[2]

            # Separar classes
            for classe, nome in zip([1, 2, 3], ['estavel', 'perda', 'ganho']):
                mascara = (dados == classe).astype(np.uint8)
                caminho_saida = os.path.join(diretorio, f"{nome}_{ano1}_{ano2}.tif")
                salvar_raster_mascara(mascara, perfil, caminho_saida)
                print(f"  Arquivo {nome} salvo: {caminho_saida}")

                # Identificar patches conectados
                binary_mask = mascara > 0
                labeled_array, num_features = label(binary_mask, structure=structure)

                # Salvar raster rotulado
                caminho_rotulado = os.path.join(diretorio, f"{nome}_patches_{ano1}_{ano2}.tif")
                perfil.update(dtype=rasterio.int32, nodata=0)
                with rasterio.open(caminho_rotulado, 'w', **perfil) as dst:
                    dst.write(labeled_array.astype(np.int32), 1)
                print(f"  Patches rotulados salvos: {caminho_rotulado}")

print("Processamento completo.")


Processando comparacao_2014_2015.tif...
  Arquivo estavel salvo: E:\Process\estavel_2014_2015.tif
  Patches rotulados salvos: E:\Process\estavel_patches_2014_2015.tif
  Arquivo perda salvo: E:\Process\perda_2014_2015.tif
  Patches rotulados salvos: E:\Process\perda_patches_2014_2015.tif
  Arquivo ganho salvo: E:\Process\ganho_2014_2015.tif
  Patches rotulados salvos: E:\Process\ganho_patches_2014_2015.tif
Processando comparacao_2015_2016.tif...
  Arquivo estavel salvo: E:\Process\estavel_2015_2016.tif
  Patches rotulados salvos: E:\Process\estavel_patches_2015_2016.tif
  Arquivo perda salvo: E:\Process\perda_2015_2016.tif
  Patches rotulados salvos: E:\Process\perda_patches_2015_2016.tif
  Arquivo ganho salvo: E:\Process\ganho_2015_2016.tif
  Patches rotulados salvos: E:\Process\ganho_patches_2015_2016.tif
Processamento completo.


In [ ]:
import os
import numpy as np
import rasterio
from scipy.ndimage import binary_dilation
from collections import defaultdict

# Diretório contendo os arquivos de patches
diretorio = r"E:\Projetos\GeoFM\Patches"

# Estrutura de conectividade rook (4-vizinhos)
structure = np.array([[0, 1, 0],
                      [1, 1, 1],
                      [0, 1, 0]])

# Função para carregar raster
def carregar_raster(caminho):
    with rasterio.open(caminho) as src:
        return src.read(1)

# Função para calcular vizinhança entre patches de A e B
def analisar_vizinhança(patches_A, patches_B):
    resultado = []
    patch_ids_A = np.unique(patches_A)
    patch_ids_A = patch_ids_A[patch_ids_A > 0]  # Ignorar fundo

    for i, patch_id in enumerate(patch_ids_A):
        mask_A = patches_A == patch_id
        dilated = binary_dilation(mask_A, structure=structure)
        border = np.logical_and(dilated, ~mask_A)

        neighbor_values = patches_B[border]
        neighbor_ids = np.unique(neighbor_values[neighbor_values > 0])

        if len(neighbor_ids) == 0:
            viz = 1
        elif len(neighbor_ids) == 1:
            if 0 in neighbor_values:
                viz = 2
            elif np.all(neighbor_values == neighbor_ids[0]):
                viz = 4
            else:
                viz = 2
        else:
            viz = 3

        resultado.append((int(patch_id), viz))

    return resultado

# Dicionário para armazenar os resultados
resultados_vizinhança = defaultdict(list)

# Processar pares de arquivos de perda/estável e ganho/estável
for nome_arquivo in os.listdir(diretorio):
    if nome_arquivo.startswith("perda_patches_") and nome_arquivo.endswith(".tif"):
        partes = nome_arquivo.replace(".tif", "").split("_")
        ano1, ano2 = partes[2], partes[3]
        caminho_perda = os.path.join(diretorio, nome_arquivo)
        caminho_estavel = os.path.join(diretorio, f"estavel_patches_{ano1}_{ano2}.tif")

        patches_A = carregar_raster(caminho_perda)
        patches_B = carregar_raster(caminho_estavel)

        resultado = analisar_vizinhança(patches_A, patches_B)
        resultados_vizinhança[f"perda_{ano1}_{ano2}"] = resultado

    elif nome_arquivo.startswith("ganho_patches_") and nome_arquivo.endswith(".tif"):
        partes = nome_arquivo.replace(".tif", "").split("_")
        ano1, ano2 = partes[2], partes[3]
        caminho_ganho = os.path.join(diretorio, nome_arquivo)
        caminho_estavel = os.path.join(diretorio, f"estavel_patches_{ano1}_{ano2}.tif")

        patches_A = carregar_raster(caminho_ganho)
        patches_B = carregar_raster(caminho_estavel)

        resultado = analisar_vizinhança(patches_A, patches_B)
        resultados_vizinhança[f"ganho_{ano1}_{ano2}"] = resultado

# Exemplo de visualização de resultados
for chave, lista in resultados_vizinhança.items():
    print(f"{chave}: {len(lista)} patches analisados")
    print("Exemplo:", lista[:5])


In [ ]:
import os
import numpy as np
import rasterio
from scipy.ndimage import binary_dilation
from collections import defaultdict

# Diretório contendo os arquivos de patches
diretorio = r"E:\Projetos\GeoFM\Patches"

# Estrutura de conectividade rook (4-vizinhos)
structure = np.array([[0, 1, 0],
                      [1, 1, 1],
                      [0, 1, 0]])

# Função para carregar raster
def carregar_raster(caminho):
    with rasterio.open(caminho) as src:
        return src.read(1)

# Função para calcular vizinhança entre patches de A e B em blocos
def analisar_vizinhança_em_blocos(patches_A, patches_B, bloco_tamanho=5000):
    resultado = []
    patch_ids_A = np.unique(patches_A)
    patch_ids_A = patch_ids_A[patch_ids_A > 0]  # Ignorar fundo
    total = len(patch_ids_A)

    for inicio in range(0, total, bloco_tamanho):
        fim = min(inicio + bloco_tamanho, total)
        bloco_ids = patch_ids_A[inicio:fim]
        print(f"Processando blocos de patches {inicio+1} a {fim} de {total}...")

        for patch_id in bloco_ids:
            mask_A = patches_A == patch_id
            dilated = binary_dilation(mask_A, structure=structure)
            border = np.logical_and(dilated, ~mask_A)

            neighbor_values = patches_B[border]
            neighbor_ids = np.unique(neighbor_values[neighbor_values > 0])

            if len(neighbor_ids) == 0:
                viz = 1
            elif len(neighbor_ids) == 1:
                if 0 in neighbor_values:
                    viz = 2
                elif np.all(neighbor_values == neighbor_ids[0]):
                    viz = 4
                else:
                    viz = 2
            else:
                viz = 3

            resultado.append((int(patch_id), viz))

    return resultado

# Dicionário para armazenar os resultados
resultados_vizinhança = defaultdict(list)

# Processar pares de arquivos de perda/estável e ganho/estável
for nome_arquivo in os.listdir(diretorio):
    if nome_arquivo.startswith("perda_patches_") and nome_arquivo.endswith(".tif"):
        partes = nome_arquivo.replace(".tif", "").split("_")
        ano1, ano2 = partes[2], partes[3]
        caminho_perda = os.path.join(diretorio, nome_arquivo)
        caminho_estavel = os.path.join(diretorio, f"estavel_patches_{ano1}_{ano2}.tif")

        patches_A = carregar_raster(caminho_perda)
        patches_B = carregar_raster(caminho_estavel)

        resultado = analisar_vizinhança_em_blocos(patches_A, patches_B)
        resultados_vizinhança[f"perda_{ano1}_{ano2}"] = resultado

    elif nome_arquivo.startswith("ganho_patches_") and nome_arquivo.endswith(".tif"):
        partes = nome_arquivo.replace(".tif", "").split("_")
        ano1, ano2 = partes[2], partes[3]
        caminho_ganho = os.path.join(diretorio, nome_arquivo)
        caminho_estavel = os.path.join(diretorio, f"estavel_patches_{ano1}_{ano2}.tif")

        patches_A = carregar_raster(caminho_ganho)
        patches_B = carregar_raster(caminho_estavel)

        resultado = analisar_vizinhança_em_blocos(patches_A, patches_B)
        resultados_vizinhança[f"ganho_{ano1}_{ano2}"] = resultado

# Exemplo de visualização de resultados
for chave, lista in resultados_vizinhança.items():
    print(f"{chave}: {len(lista)} patches analisados")
    print("Exemplo:", lista[:5])


In [ ]:
import os
import numpy as np
import rasterio

diretorio = r"E:\Process"

print("Convertendo arquivos GeoTIFF para .npy...")
for nome_arquivo in os.listdir(diretorio):
    if nome_arquivo.endswith(".tif") and "_patches_" in nome_arquivo:
        caminho_tif = os.path.join(diretorio, nome_arquivo)
        caminho_npy = os.path.join(diretorio, nome_arquivo.replace(".tif", ".npy"))
        with rasterio.open(caminho_tif) as src:
            array = src.read(1)
            np.save(caminho_npy, array)
        print(f"  Convertido: {nome_arquivo} -> {os.path.basename(caminho_npy)}")
print("Conversão concluída.\n")


In [ ]:
from scipy.ndimage import binary_dilation
from collections import defaultdict

structure = np.array([[0, 1, 0],
                      [1, 1, 1],
                      [0, 1, 0]])

def analisar_vizinhança_em_blocos(patches_A, patches_B, bloco_tamanho=5000):
    resultado = []
    patch_ids_A = np.unique(patches_A)
    patch_ids_A = patch_ids_A[patch_ids_A > 0]
    total = len(patch_ids_A)

    for inicio in range(0, total, bloco_tamanho):
        fim = min(inicio + bloco_tamanho, total)
        bloco_ids = patch_ids_A[inicio:fim]
        print(f"  Processando blocos de patches {inicio+1} a {fim} de {total}...")

        for patch_id in bloco_ids:
            mask_A = patches_A == patch_id
            dilated = binary_dilation(mask_A, structure=structure)
            border = np.logical_and(dilated, ~mask_A)

            neighbor_values = patches_B[border]
            neighbor_ids = np.unique(neighbor_values[neighbor_values > 0])

            if len(neighbor_ids) == 0:
                viz = 1
            elif len(neighbor_ids) == 1:
                if 0 in neighbor_values:
                    viz = 2
                elif np.all(neighbor_values == neighbor_ids[0]):
                    viz = 4
                else:
                    viz = 2
            else:
                viz = 3

            resultado.append((int(patch_id), viz))

    return resultado

# Executar análise
resultados_vizinhança = defaultdict(list)

print("Iniciando análise de vizinhança com arquivos .npy...\n")
for nome_arquivo in os.listdir(diretorio):
    if nome_arquivo.startswith("perda_patches_") and nome_arquivo.endswith(".npy"):
        partes = nome_arquivo.replace(".npy", "").split("_")
        ano1, ano2 = partes[2], partes[3]
        caminho_perda = os.path.join(diretorio, nome_arquivo)
        caminho_estavel = os.path.join(diretorio, f"estavel_patches_{ano1}_{ano2}.npy")

        if os.path.exists(caminho_estavel):
            patches_A = np.load(caminho_perda)
            patches_B = np.load(caminho_estavel)
            print(f"Analisando perda_{ano1}_{ano2}...")
            resultado = analisar_vizinhança_em_blocos(patches_A, patches_B)
            resultados_vizinhança[f"perda_{ano1}_{ano2}"] = resultado

    elif nome_arquivo.startswith("ganho_patches_") and nome_arquivo.endswith(".npy"):
        partes = nome_arquivo.replace(".npy", "").split("_")
        ano1, ano2 = partes[2], partes[3]
        caminho_ganho = os.path.join(diretorio, nome_arquivo)
        caminho_estavel = os.path.join(diretorio, f"estavel_patches_{ano1}_{ano2}.npy")

        if os.path.exists(caminho_estavel):
            patches_A = np.load(caminho_ganho)
            patches_B = np.load(caminho_estavel)
            print(f"Analisando ganho_{ano1}_{ano2}...")
            resultado = analisar_vizinhança_em_blocos(patches_A, patches_B)
            resultados_vizinhança[f"ganho_{ano1}_{ano2}"] = resultado

print("\nAnálise de vizinhança concluída.")
for chave, lista in resultados_vizinhança.items():
    print(f"{chave}: {len(lista)} patches analisados. Exemplo: {lista[:5]}")


In [1]:
import os
import numpy as np
import time
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor
from skimage.morphology import binary_dilation

# Diretório contendo os arquivos .npy
diretorio = r"E:\Projetos\GeoFM\Patches"

# Estrutura de conectividade rook (4-vizinhos)
structure = np.array([[0, 1, 0],
                      [1, 1, 1],
                      [0, 1, 0]])

# Função para processar um único patch
def processar_patch(patch_id, patches_A, patches_B):
    mask_A = patches_A == patch_id
    dilated = binary_dilation(mask_A, footprint=structure)
    border = np.logical_and(dilated, ~mask_A)

    neighbor_values = patches_B[border]
    neighbor_ids = np.unique(neighbor_values[neighbor_values > 0])

    if len(neighbor_ids) == 0:
        viz = 1
    elif len(neighbor_ids) == 1:
        if 0 in neighbor_values:
            viz = 2
        elif np.all(neighbor_values == neighbor_ids[0]):
            viz = 4
        else:
            viz = 2
    else:
        viz = 3

    return (int(patch_id), viz)

# Função para processar blocos de patches com paralelismo
def analisar_vizinhança_em_blocos(patches_A, patches_B, bloco_tamanho=5000):
    resultado = []
    patch_ids_A = np.unique(patches_A)
    patch_ids_A = patch_ids_A[patch_ids_A > 0]
    total = len(patch_ids_A)

    for inicio in range(0, total, bloco_tamanho):
        fim = min(inicio + bloco_tamanho, total)
        bloco_ids = patch_ids_A[inicio:fim]
        print(f"  Processando blocos de patches {inicio+1} a {fim} de {total}...")
        bloco_start = time.time()

        with ThreadPoolExecutor() as executor:
            args = ((patch_id, patches_A, patches_B) for patch_id in bloco_ids)
            bloco_resultado = list(executor.map(lambda p: processar_patch(*p), args))

        resultado.extend(bloco_resultado)
        print(f"    Tempo para bloco: {time.time() - bloco_start:.2f} segundos")

    return resultado

# Executar análise
resultados_vizinhança = defaultdict(list)

print("Iniciando análise de vizinhança com arquivos .npy...\n")
for nome_arquivo in os.listdir(diretorio):
    if nome_arquivo.startswith("perda_patches_") and nome_arquivo.endswith(".npy"):
        partes = nome_arquivo.replace(".npy", "").split("_")
        ano1, ano2 = partes[2], partes[3]
        caminho_perda = os.path.join(diretorio, nome_arquivo)
        caminho_estavel = os.path.join(diretorio, f"estavel_patches_{ano1}_{ano2}.npy")

        if os.path.exists(caminho_estavel):
            patches_A = np.load(caminho_perda)
            patches_B = np.load(caminho_estavel)
            print(f"Analisando perda_{ano1}_{ano2}...")
            resultado = analisar_vizinhança_em_blocos(patches_A, patches_B)
            resultados_vizinhança[f"perda_{ano1}_{ano2}"] = resultado

    elif nome_arquivo.startswith("ganho_patches_") and nome_arquivo.endswith(".npy"):
        partes = nome_arquivo.replace(".npy", "").split("_")
        ano1, ano2 = partes[2], partes[3]
        caminho_ganho = os.path.join(diretorio, nome_arquivo)
        caminho_estavel = os.path.join(diretorio, f"estavel_patches_{ano1}_{ano2}.npy")

        if os.path.exists(caminho_estavel):
            patches_A = np.load(caminho_ganho)
            patches_B = np.load(caminho_estavel)
            print(f"Analisando ganho_{ano1}_{ano2}...")
            resultado = analisar_vizinhança_em_blocos(patches_A, patches_B)
            resultados_vizinhança[f"ganho_{ano1}_{ano2}"] = resultado

print("\nAnálise de vizinhança concluída.")
for chave, lista in resultados_vizinhança.items():
    print(f"{chave}: {len(lista)} patches analisados. Exemplo: {lista[:5]}")


Iniciando análise de vizinhança com arquivos .npy...

Analisando ganho_1985_1986...
  Processando blocos de patches 1 a 5000 de 165466...
    Tempo para bloco: 401.23 segundos
  Processando blocos de patches 5001 a 10000 de 165466...
    Tempo para bloco: 430.91 segundos
  Processando blocos de patches 10001 a 15000 de 165466...
    Tempo para bloco: 434.05 segundos
  Processando blocos de patches 15001 a 20000 de 165466...
    Tempo para bloco: 432.07 segundos
  Processando blocos de patches 20001 a 25000 de 165466...
    Tempo para bloco: 428.65 segundos
  Processando blocos de patches 25001 a 30000 de 165466...
    Tempo para bloco: 427.80 segundos
  Processando blocos de patches 30001 a 35000 de 165466...
    Tempo para bloco: 429.81 segundos
  Processando blocos de patches 35001 a 40000 de 165466...
    Tempo para bloco: 431.70 segundos
  Processando blocos de patches 40001 a 45000 de 165466...
    Tempo para bloco: 434.44 segundos
  Processando blocos de patches 45001 a 50000 de 

KeyboardInterrupt: 

In [2]:
pip install cupy-cuda12x


   ---------------------------------------- 0.0/82.1 MB ? eta -:--:--
    --------------------------------------- 1.8/82.1 MB 12.6 MB/s eta 0:00:07
   -- ------------------------------------- 5.8/82.1 MB 16.8 MB/s eta 0:00:05
   ----- ---------------------------------- 10.5/82.1 MB 18.7 MB/s eta 0:00:04
   ------ --------------------------------- 14.2/82.1 MB 18.5 MB/s eta 0:00:04
   ---------- ----------------------------- 21.0/82.1 MB 22.1 MB/s eta 0:00:03
   ----------- ---------------------------- 24.4/82.1 MB 21.4 MB/s eta 0:00:03
   --------------- ------------------------ 30.9/82.1 MB 22.3 MB/s eta 0:00:03
   ----------------- ---------------------- 36.7/82.1 MB 23.3 MB/s eta 0:00:02
   -------------------- ------------------- 42.7/82.1 MB 24.0 MB/s eta 0:00:02
   ----------------------- ---------------- 49.0/82.1 MB 24.6 MB/s eta 0:00:02
   --------------------------- ------------ 56.1/82.1 MB 25.3 MB/s eta 0:00:02
   ------------------------------ --------- 62.7/82.1 MB 25.9 M

In [3]:
import cupy as cp

print("Número de GPUs disponíveis:", cp.cuda.runtime.getDeviceCount())
print("Nome da GPU:", cp.cuda.Device(0).name)


C:\Users\mario.barroso\AppData\Local\anaconda3\Lib\site-packages\cupy\_environment.py:217: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


Número de GPUs disponíveis: 1


AttributeError: 'cupy.cuda.device.Device' object has no attribute 'name'

In [ ]:
import os
import time
import cupy as cp
import numpy as np
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor
from cupyx.scipy.ndimage import binary_dilation

# Diretório contendo os arquivos .npy
diretorio = r"E:\Process"

# Estrutura de conectividade rook (4-vizinhos) em CuPy
structure = cp.array([[0, 1, 0],
                      [1, 1, 1],
                      [0, 1, 0]], dtype=cp.uint8)

# Função para processar um único patch na GPU
def processar_patch_gpu(patch_id, patches_A_gpu, patches_B_gpu):
    mask_A = patches_A_gpu == patch_id
    dilated = binary_dilation(mask_A, structure=structure)
    border = cp.logical_and(dilated, ~mask_A)

    neighbor_values = patches_B_gpu[border]
    neighbor_ids = cp.unique(neighbor_values[neighbor_values > 0])

    if len(neighbor_ids) == 0:
        viz = 1
    elif len(neighbor_ids) == 1:
        if cp.any(neighbor_values == 0):
            viz = 2
        elif cp.all(neighbor_values == neighbor_ids[0]):
            viz = 4
        else:
            viz = 2
    else:
        viz = 3

    return int(patch_id), viz

# Função para processar blocos de patches com paralelismo
def analisar_vizinhança_em_blocos_gpu(patches_A, patches_B, bloco_tamanho=5000):
    resultado = []
    patch_ids_A = cp.unique(patches_A)
    patch_ids_A = patch_ids_A[patch_ids_A > 0]
    total = len(patch_ids_A)

    for inicio in range(0, total, bloco_tamanho):
        fim = min(inicio + bloco_tamanho, total)
        bloco_ids = patch_ids_A[inicio:fim]
        print(f"  Processando blocos de patches {inicio+1} a {fim} de {total}...")
        bloco_start = time.time()

        # Transferir bloco de IDs para CPU para paralelismo
        bloco_ids_cpu = cp.asnumpy(bloco_ids)
        with ThreadPoolExecutor() as executor:
            args = ((patch_id, patches_A, patches_B) for patch_id in bloco_ids_cpu)
            bloco_resultado = list(executor.map(lambda p: processar_patch_gpu(*p), args))

        resultado.extend(bloco_resultado)
        print(f"    Tempo para bloco: {time.time() - bloco_start:.2f} segundos")

    return resultado

# Executar análise
resultados_vizinhança = defaultdict(list)

print("Iniciando análise de vizinhança com CuPy e GPU...\n")
for nome_arquivo in os.listdir(diretorio):
    if nome_arquivo.startswith("perda_patches_") and nome_arquivo.endswith(".npy"):
        partes = nome_arquivo.replace(".npy", "").split("_")
        ano1, ano2 = partes[2], partes[3]
        caminho_perda = os.path.join(diretorio, nome_arquivo)
        caminho_estavel = os.path.join(diretorio, f"estavel_patches_{ano1}_{ano2}.npy")

        if os.path.exists(caminho_estavel):
            patches_A = cp.load(caminho_perda)
            patches_B = cp.load(caminho_estavel)
            print(f"Analisando perda_{ano1}_{ano2}...")
            resultado = analisar_vizinhança_em_blocos_gpu(patches_A, patches_B)
            resultados_vizinhança[f"perda_{ano1}_{ano2}"] = resultado

    elif nome_arquivo.startswith("ganho_patches_") and nome_arquivo.endswith(".npy"):
        partes = nome_arquivo.replace(".npy", "").split("_")
        ano1, ano2 = partes[2], partes[3]
        caminho_ganho = os.path.join(diretorio, nome_arquivo)
        caminho_estavel = os.path.join(diretorio, f"estavel_patches_{ano1}_{ano2}.npy")

        if os.path.exists(caminho_estavel):
            patches_A = cp.load(caminho_ganho)
            patches_B = cp.load(caminho_estavel)
            print(f"Analisando ganho_{ano1}_{ano2}...")
            resultado = analisar_vizinhança_em_blocos_gpu(patches_A, patches_B)
            resultados_vizinhança[f"ganho_{ano1}_{ano2}"] = resultado

print("\nAnálise de vizinhança concluída.")
for chave, lista in resultados_vizinhança.items():
    print(f"{chave}: {len(lista)} patches analisados. Exemplo: {lista[:5]}")
